# Process
FIFA happens in 2 stages, the group stage, & knockout stage. For men 16 teams advance of the 32 to the knockout stage.

### Group Stage
In the group stage, the teams that qualified for the tournament (32 for men, 24 for women) are broken into groups containing four teams each. Each team competes in a round robin tournament with the other three teams in its group.

The two top teams in each group advance to the Knockout Stage. 

### Knockout Stage

The remaining 16 teams participate in a single elimination bracket pool. For those of you familiar with the NCAA Basketball tournament, think of the Knockout round as the soccer equivalent to the Sweet 16.

## Process

Starting at the end... What are we trying to predict? Team strength. Team win rate. We can find this in the results.csv

We have *player features* found in `player_features.csv`, *players by teams* in `players_by_teams.csv`, *team results* in `team_results.csv`

### multiclass problem

win/lose/draw?
sci kit learn to split data into multiple groups. Train, Test. 

Use ensamble to refine other things

1. logistic regression
1. support vector machine
1. xgboost


In [2]:
import xgboost as xgb
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display
pd.set_option('display.max_rows', 500)

In [3]:
pf = pd.read_csv('./data/player_features.csv')
tr = pd.read_csv('./data/team_results.csv')
pbt = pd.read_csv('./data/players_by_team.csv')

display(pf.head())
display(tr.head())
display(pbt.head())

,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes
0,Cristiano Ronaldo,Portugal,LS,7.0,Real Madrid,LW,7.0,07/01/2009,2021.0,94,...,90,81,76,85,88,14,7,15,11,11
1,Lionel Messi,Argentina,RW,10.0,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,...,88,89,90,74,85,14,6,15,11,8
2,Neymar,Brazil,LW,10.0,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,...,77,79,84,81,83,15,9,15,9,11
3,Luis Suárez,Uruguay,LS,9.0,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,...,86,86,84,85,88,33,27,31,25,37
4,Manuel Neuer,Germany,GK,1.0,FC Bayern,GK,1.0,07/01/2011,2021.0,92,...,16,14,11,47,11,91,89,95,90,89


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


,Name,FIFA_POS,Country
0,Fahad Al-Muwallad,FW,Saudia Arabia
1,Mohammad Al-Sahlawi,FW,Saudia Arabia
2,Osama Hawsawi,DF,Saudia Arabia
3,Yahya Al-Shehri,Midfielder,Saudia Arabia
4,Taisir Al-Jassim,Midfielder,Saudia Arabia


In [4]:
# get final data:
# final_data = Name, Nationality, FIFA_POS, National_Position,National_Kit, Club, Club_Position, Club_Kit, Club_Joining, Contract_Expiry, Rating, Long_Shots, Curve, Freekick_Accuracy, Penalties, Volleys, GK_Positioning, GK_Diving, GK_Kicking, GK_Handling, GK_Reflexes

#1. Drop Data From Player_Features, that aren't playing.
an=pbt.iloc[:,0:1].values # an = athlete names
an = an.flatten()
athlete_data_names = pf.iloc[:,0:1].values
athlete_data_names = athlete_data_names.flatten()

#1.5 Find players not listed in pf before dropping
missing_players = pbt[~pbt['Name'].isin(athlete_data_names)]
print("Missing Players")
print(missing_players.shape[0])
display(missing_players.head())

pf = pf[pf['Name'].isin(an)]

#2. Append FIFA_POS & Country
pf = pd.merge(pf, pbt, on=['Name'])

print("Player Features Merged")
display(pf.head())

#should be 736 (23players * 32teams)
print(pf.shape[0])
print("Should be 736...")

Missing Players
231


,Name,FIFA_POS,Country
0,Fahad Al-Muwallad,FW,Saudia Arabia
1,Mohammad Al-Sahlawi,FW,Saudia Arabia
3,Yahya Al-Shehri,Midfielder,Saudia Arabia
4,Taisir Al-Jassim,Midfielder,Saudia Arabia
5,Salem Al-Dawsari,Midfielder,Saudia Arabia


Player Features Merged


,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,FIFA_POS,Country
0,Cristiano Ronaldo,Portugal,LS,7.0,Real Madrid,LW,7.0,07/01/2009,2021.0,94,...,76,85,88,14,7,15,11,11,FW,Portugal
1,Lionel Messi,Argentina,RW,10.0,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,...,90,74,85,14,6,15,11,8,FW,Argentina
2,Neymar,Brazil,LW,10.0,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,...,84,81,83,15,9,15,9,11,FW,Brazil
3,Luis Suárez,Uruguay,LS,9.0,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,...,84,85,88,33,27,31,25,37,FW,Uruguay
4,Manuel Neuer,Germany,GK,1.0,FC Bayern,GK,1.0,07/01/2011,2021.0,92,...,11,47,11,91,89,95,90,89,GK,Germany


522
Should be 736...


In [5]:
# Find out which teams are missing data
# Get count of players in each team
player_by_country = pf['Country'].value_counts()
print(player_by_country)

Brazil            27
England           24
Germany           23
Argentina         23
Poland            22
Switzerland       21
Belgium           21
France            21
Portugal          20
Mexico            20
Denmark           20
Sweden            20
Australia         19
Croatia           18
Spain             18
Serbia            18
Senegal           18
Japan             17
Colombia          16
Peru              16
Morocco           16
Iceland           14
Nigeria           13
Egypt             13
Uruguay           12
Costa Rica        11
Russia            10
Iran               9
Tunisia            9
Panama             7
Saudia Arabia      4
Korea Republic     2
Name: Country, dtype: int64


In [7]:
# print out any interesting countries that need more data....df.loc[df['column_name'] == some_value]
display(pf.loc[pf['Country'] == 'Brazil'])

,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,FIFA_POS,Country
2,Neymar,Brazil,LW,10.0,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,...,84,81,83,15,9,15,9,11,FW,Brazil
10,Thiago Silva,Brazil,Sub,14.0,PSG,LCB,2.0,07/01/2012,2020.0,89,...,73,71,63,9,9,5,12,10,DF,Brazil
32,Marcelo,Brazil,LB,16.0,Real Madrid,LB,12.0,01/01/2007,2020.0,86,...,67,59,54,5,12,5,5,9,DF,Brazil
33,Marcelo,Brazil,NaN,NaN,Rio Ave,RCB,46.0,07/01/2012,2019.0,77,...,27,42,28,10,6,6,16,8,DF,Brazil
34,Marcelo,Brazil,NaN,NaN,Beşiktaş,RCB,30.0,07/01/2013,2019.0,77,...,41,51,42,7,16,10,15,14,DF,Brazil
35,Miranda,Brazil,RCB,3.0,Inter,RCB,25.0,07/03/2015,2018.0,86,...,39,49,51,13,12,10,6,12,DF,Brazil
41,Willian,Brazil,Sub,19.0,Chelsea,Sub,22.0,08/28/2013,2020.0,85,...,85,60,59,7,14,13,12,13,MF,Brazil
44,Filipe Luís,Brazil,Sub,6.0,Atlético Madrid,LB,3.0,07/29/2015,2019.0,85,...,75,65,56,9,11,9,15,5,DF,Brazil
54,Douglas Costa,Brazil,Sub,7.0,FC Bayern,LM,11.0,07/01/2015,2020.0,84,...,78,77,73,12,13,9,15,5,FW,Brazil
64,Marquinhos,Brazil,LCB,13.0,PSG,RCB,5.0,07/19/2013,2019.0,83,...,32,31,31,10,6,12,6,13,DF,Brazil


In [8]:
#Drop all matches before 2008
from datetime import datetime

def get_year(row):
    return datetime.strptime(row['date'], '%Y-%m-%d').year

tr['year_played'] = tr.apply (lambda row: get_year (row),axis=1)

tr = tr[(tr['year_played'] > 2008)]

In [9]:
# Drop all matches, where we won't have both teams in the world cup
from matches_groups import FIFA_teams 

teams = FIFA_teams()
print(str(tr.shape[0]) + " games tracked.")
print("removing games that don't involve both teams in FIFA")
# missing_players = pbt[~pbt['Name'].isin(athlete_data_names)]
# pf = pf[pf['Name'].isin(an)]

tr = tr[(tr['home_team'].isin(teams)) & (tr['away_team'].isin(teams))]
print(str(tr.shape[0]) + " games tracked.")

display(tr.head(10))



8487 games tracked.
removing games that don't involve both teams in FIFA
615 games tracked.


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_played
30613,2009-01-23,Costa Rica,Panama,3,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009
30629,2009-01-28,Mexico,Sweden,0,1,Friendly,Oakland,USA,True,2009
30638,2009-02-01,Costa Rica,Panama,0,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009
30663,2009-02-11,France,Argentina,0,2,Friendly,Marseille,France,False,2009
30666,2009-02-11,Iran,Korea Republic,1,1,FIFA World Cup qualification,Teheran,Iran,False,2009
30669,2009-02-11,Japan,Australia,0,0,FIFA World Cup qualification,Yokohama,Japan,False,2009
30686,2009-02-11,Spain,England,2,0,Friendly,Seville,Spain,False,2009
30729,2009-03-28,Iran,Saudi Arabia,1,2,FIFA World Cup qualification,Teheran,Iran,False,2009
30739,2009-03-28,Mexico,Costa Rica,2,0,FIFA World Cup qualification,Mexico City,Mexico,False,2009
30747,2009-03-28,Portugal,Sweden,0,0,FIFA World Cup qualification,Porto,Portugal,False,2009


In [10]:
# Now let's fix the team_record aka tr
# date	home_team	away_team	home_score	away_score	tournament	city	country	neutral
tr['PT_DIFF'] = abs(tr['home_score'] - tr['away_score'])
def label_result (row):
    if row['home_score'] - row['away_score'] < 0 :
        return 'A'
    if row['home_score'] - row['away_score'] > 0 :
        return 'H'
    return 'T'

tr['FTR'] = tr.apply (lambda row: label_result (row),axis=1)

display(tr.head(10))

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_played,PT_DIFF,FTR
30613,2009-01-23,Costa Rica,Panama,3,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009,3,H
30629,2009-01-28,Mexico,Sweden,0,1,Friendly,Oakland,USA,True,2009,1,A
30638,2009-02-01,Costa Rica,Panama,0,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009,0,T
30663,2009-02-11,France,Argentina,0,2,Friendly,Marseille,France,False,2009,2,A
30666,2009-02-11,Iran,Korea Republic,1,1,FIFA World Cup qualification,Teheran,Iran,False,2009,0,T
30669,2009-02-11,Japan,Australia,0,0,FIFA World Cup qualification,Yokohama,Japan,False,2009,0,T
30686,2009-02-11,Spain,England,2,0,Friendly,Seville,Spain,False,2009,2,H
30729,2009-03-28,Iran,Saudi Arabia,1,2,FIFA World Cup qualification,Teheran,Iran,False,2009,1,A
30739,2009-03-28,Mexico,Costa Rica,2,0,FIFA World Cup qualification,Mexico City,Mexico,False,2009,2,H
30747,2009-03-28,Portugal,Sweden,0,0,FIFA World Cup qualification,Porto,Portugal,False,2009,0,T


In [11]:
#Explore: Show how many games each team played from FIFA_teams()
htg = tr['home_team'].value_counts()
atg = tr['away_team'].value_counts()

# Country | home showings | away showings | n showings | wins | win%
columns = ["TeamNum","Country","home showings","away showings","n showings"]
team_num = 1

team_arr = []

for country in teams:  
    team_arr.append([ team_num, country, htg[country], atg[country], htg[country] + atg[country] ])
    team_num += 1

    
explorer = pd.DataFrame(team_arr, columns=columns)
total_showings = explorer['n showings'].sum()
explorer['%shown'] = explorer["n showings"] / total_showings * 100

display(explorer)

,TeamNum,Country,home showings,away showings,n showings,%shown
0,1,Uruguay,19,30,49,3.983740
1,2,Egypt,9,9,18,1.463415
2,3,Saudi Arabia,8,15,23,1.869919
3,4,Russia,25,9,34,2.764228
4,5,Iran,15,8,23,1.869919
5,6,Spain,24,22,46,3.739837
6,7,Morocco,7,7,14,1.138211
7,8,Portugal,22,25,47,3.821138
8,9,Australia,18,22,40,3.252033
9,10,Denmark,26,12,38,3.089431


In [12]:
# Visualize data
rows =  tr.shape[0]
home_wins = len(tr[tr.FTR == "H"])
hWinRate = home_wins / rows * 100
print("Home Win Rate:" + str(hWinRate) + "%")

Home Win Rate:46.016260162601625%


In [13]:
# Define some convience functions
def get_team_max_min_avg(feature, dataframe):
    """Return MAX,MIN,AVG for a feature for a team"""
    if(feature == 'Height'):
        dataframe['Height'] = dataframe['Height'][:-3]
    if(feature == 'Weight'):
        dataframe['Weight'] = dataframe['Weight'][:-3]
        
    return [dataframe[feature].max(), dataframe[feature].min(), dataframe[feature].mean()]

def get_pf_on_team(team):
    """Return Dataframe of player features"""
    return pf.loc[pf['Country'] == team]

def add_maxminmean_vals_to_df(df, vals):
    for val in vals:
        df[str(val)+'_max'] = 0
        df[str(val)+'_min'] = 0
        df[str(val)+'_avg'] = 0
    
    return df

def add_home_away_maxminmean_vals_to_df(df, vals, home):
    if home:
        for val in vals:
            df['ho_'+str(val)+'_max'] = 0
            df['ho_'+str(val)+'_min'] = 0
            df['ho_'+str(val)+'_avg'] = 0
    else:
        for val in vals:
            df['aw_'+str(val)+'_max'] = 0
            df['aw_'+str(val)+'_min'] = 0
            df['aw_'+str(val)+'_avg'] = 0
    
    return df


In [14]:
# Testing
df = get_pf_on_team('Saudi Arabia')
display(df)
print(get_team_max_min_avg('Rating', df))


,Name,Nationality,National_Position,National_Kit,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,...,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,FIFA_POS,Country


[nan, nan, nan]


In [15]:
# Add a whole bunch of features to our teams on the explorer & fill with nan values
# extra_columns = ['Rating','Height','Weight','Age','Skill_Moves','Ball_Control','Dribbling','Marking','Sliding_Tackle','Standing_Tackle','Aggression','Reactions','Attacking_Position','Interceptions','Vision','Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina','Strength','Balance','Agility','Jumping','Heading','Shot_Power','Finishing','Long_Shots','Curve','Freekick_Accuracy','Penalties','Volleys','GK_Positioning','GK_Diving','GK_Kicking','GK_Handling','GK_Reflexes']
extra_columns = ['Rating','Age','Skill_Moves','Ball_Control','Dribbling','Marking','Sliding_Tackle','Standing_Tackle','Aggression','Reactions','Attacking_Position','Interceptions','Vision','Composure','Crossing','Short_Pass','Long_Pass','Acceleration','Speed','Stamina','Strength','Balance','Agility','Jumping','Heading','Shot_Power','Finishing','Long_Shots','Curve','Freekick_Accuracy','Penalties','Volleys','GK_Positioning','GK_Diving','GK_Kicking','GK_Handling','GK_Reflexes']

explorer = add_maxminmean_vals_to_df(explorer, extra_columns)

# display(explorer)

for team in teams:
    if team == 'Saudi Arabia':
        for col in extra_columns:
            rtr = [50,50,50]
            explorer.loc[explorer['Country'] == team, col+'_max'] = rtr[0]
            explorer.loc[explorer['Country'] == team, col+'_min'] = rtr[1]
            explorer.loc[explorer['Country'] == team, col+'_avg'] = rtr[2]
        
    else:    
        team_pf = get_pf_on_team(team)
        for col in extra_columns:
            rtr = get_team_max_min_avg(col, team_pf)
            explorer.loc[explorer['Country'] == team, col+'_max'] = rtr[0]
            explorer.loc[explorer['Country'] == team, col+'_min'] = rtr[1]
            explorer.loc[explorer['Country'] == team, col+'_avg'] = rtr[2]
        
    
display(explorer)

,TeamNum,Country,home showings,away showings,n showings,%shown,Rating_max,Rating_min,Rating_avg,Age_max,...,GK_Diving_avg,GK_Kicking_max,GK_Kicking_min,GK_Kicking_avg,GK_Handling_max,GK_Handling_min,GK_Handling_avg,GK_Reflexes_max,GK_Reflexes_min,GK_Reflexes_avg
0,1,Uruguay,19,30,49,3.983740,92,69,78.250000,31,...,16.333333,71,4,18.833333,75,3,15.333333,81,4,18.250000
1,2,Egypt,9,9,18,1.463415,82,65,71.076923,44,...,20.769231,77,8,21.153846,72,7,20.384615,79,6,20.000000
2,3,Saudi Arabia,8,15,23,1.869919,50,50,50.000000,50,...,50.000000,50,50,50.000000,50,50,50.000000,50,50,50.000000
3,4,Russia,25,9,34,2.764228,81,68,75.900000,33,...,24.300000,78,10,25.100000,73,6,22.700000,83,6,24.200000
4,5,Iran,15,8,23,1.869919,77,62,70.555556,30,...,15.666667,59,6,17.000000,60,7,16.000000,63,7,17.111111
5,6,Spain,24,22,46,3.739837,89,60,78.166667,31,...,13.333333,56,1,11.444444,64,1,13.555556,63,1,12.611111
6,7,Morocco,7,7,14,1.138211,84,69,74.625000,32,...,14.750000,69,6,13.937500,78,4,14.937500,64,6,13.250000
7,8,Portugal,22,25,47,3.821138,94,64,80.650000,35,...,21.650000,80,5,20.750000,81,5,20.550000,87,5,20.850000
8,9,Australia,18,22,40,3.252033,78,57,70.157895,37,...,20.578947,81,7,20.526316,75,7,20.526316,79,6,21.157895
9,10,Denmark,26,12,38,3.089431,85,67,75.050000,33,...,16.800000,73,6,16.800000,79,6,17.350000,77,6,17.000000


In [16]:
team1 = 6
team2 = 8
team3 = 20
team4 = 23



pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(explorer.loc[explorer['TeamNum']==team1])
display(explorer.loc[explorer['TeamNum']==team2])
display(explorer.loc[explorer['TeamNum']==team3])
display(explorer.loc[explorer['TeamNum']==team4])


height has been deprecated.



,TeamNum,Country,home showings,away showings,n showings,%shown,Rating_max,Rating_min,Rating_avg,Age_max,Age_min,Age_avg,Skill_Moves_max,Skill_Moves_min,Skill_Moves_avg,Ball_Control_max,Ball_Control_min,Ball_Control_avg,Dribbling_max,Dribbling_min,Dribbling_avg,Marking_max,Marking_min,Marking_avg,Sliding_Tackle_max,Sliding_Tackle_min,Sliding_Tackle_avg,Standing_Tackle_max,Standing_Tackle_min,Standing_Tackle_avg,Aggression_max,Aggression_min,Aggression_avg,Reactions_max,Reactions_min,Reactions_avg,Attacking_Position_max,Attacking_Position_min,Attacking_Position_avg,Interceptions_max,Interceptions_min,Interceptions_avg,Vision_max,Vision_min,Vision_avg,Composure_max,Composure_min,Composure_avg,Crossing_max,Crossing_min,Crossing_avg,Short_Pass_max,Short_Pass_min,Short_Pass_avg,Long_Pass_max,Long_Pass_min,Long_Pass_avg,Acceleration_max,Acceleration_min,Acceleration_avg,Speed_max,Speed_min,Speed_avg,Stamina_max,Stamina_min,Stamina_avg,Strength_max,Strength_min,Strength_avg,Balance_max,Balance_min,Balance_avg,Agility_max,Agility_min,Agility_avg,Jumping_max,Jumping_min,Jumping_avg,Heading_max,Heading_min,Heading_avg,Shot_Power_max,Shot_Power_min,Shot_Power_avg,Finishing_max,Finishing_min,Finishing_avg,Long_Shots_max,Long_Shots_min,Long_Shots_avg,Curve_max,Curve_min,Curve_avg,Freekick_Accuracy_max,Freekick_Accuracy_min,Freekick_Accuracy_avg,Penalties_max,Penalties_min,Penalties_avg,Volleys_max,Volleys_min,Volleys_avg,GK_Positioning_max,GK_Positioning_min,GK_Positioning_avg,GK_Diving_max,GK_Diving_min,GK_Diving_avg,GK_Kicking_max,GK_Kicking_min,GK_Kicking_avg,GK_Handling_max,GK_Handling_min,GK_Handling_avg,GK_Reflexes_max,GK_Reflexes_min,GK_Reflexes_avg
5,6,Spain,24,22,46,3.739837,89,60,78.166667,31,20,26.333333,4,1,3.0,90,10,75.0,89,11,71.0,85,13,46.611111,90,10,50.944444,89,12,55.833333,93,15,64.0,86,49,74.055556,88,10,67.055556,88,12,54.555556,92,42,69.611111,92,28,67.666667,85,13,66.777778,91,12,72.166667,85,20,66.333333,93,22,73.5,93,25,70.888889,90,29,73.722222,91,49,64.611111,88,42,71.166667,92,39,73.777778,92,52,67.722222,90,15,57.611111,84,21,67.777778,89,12,61.833333,82,15,63.333333,86,13,69.166667,77,12,61.222222,82,11,60.055556,86,15,56.722222,61,1,11.944444,65,1,13.333333,56,1,11.444444,64,1,13.555556,63,1,12.611111


,TeamNum,Country,home showings,away showings,n showings,%shown,Rating_max,Rating_min,Rating_avg,Age_max,Age_min,Age_avg,Skill_Moves_max,Skill_Moves_min,Skill_Moves_avg,Ball_Control_max,Ball_Control_min,Ball_Control_avg,Dribbling_max,Dribbling_min,Dribbling_avg,Marking_max,Marking_min,Marking_avg,Sliding_Tackle_max,Sliding_Tackle_min,Sliding_Tackle_avg,Standing_Tackle_max,Standing_Tackle_min,Standing_Tackle_avg,Aggression_max,Aggression_min,Aggression_avg,Reactions_max,Reactions_min,Reactions_avg,Attacking_Position_max,Attacking_Position_min,Attacking_Position_avg,Interceptions_max,Interceptions_min,Interceptions_avg,Vision_max,Vision_min,Vision_avg,Composure_max,Composure_min,Composure_avg,Crossing_max,Crossing_min,Crossing_avg,Short_Pass_max,Short_Pass_min,Short_Pass_avg,Long_Pass_max,Long_Pass_min,Long_Pass_avg,Acceleration_max,Acceleration_min,Acceleration_avg,Speed_max,Speed_min,Speed_avg,Stamina_max,Stamina_min,Stamina_avg,Strength_max,Strength_min,Strength_avg,Balance_max,Balance_min,Balance_avg,Agility_max,Agility_min,Agility_avg,Jumping_max,Jumping_min,Jumping_avg,Heading_max,Heading_min,Heading_avg,Shot_Power_max,Shot_Power_min,Shot_Power_avg,Finishing_max,Finishing_min,Finishing_avg,Long_Shots_max,Long_Shots_min,Long_Shots_avg,Curve_max,Curve_min,Curve_avg,Freekick_Accuracy_max,Freekick_Accuracy_min,Freekick_Accuracy_avg,Penalties_max,Penalties_min,Penalties_avg,Volleys_max,Volleys_min,Volleys_avg,GK_Positioning_max,GK_Positioning_min,GK_Positioning_avg,GK_Diving_max,GK_Diving_min,GK_Diving_avg,GK_Kicking_max,GK_Kicking_min,GK_Kicking_avg,GK_Handling_max,GK_Handling_min,GK_Handling_avg,GK_Reflexes_max,GK_Reflexes_min,GK_Reflexes_avg
7,8,Portugal,22,25,47,3.821138,94,64,80.65,35,20,26.85,5,1,2.75,93,18,69.45,92,14,68.15,88,13,51.95,89,10,50.55,87,13,55.4,94,29,65.4,96,63,78.9,94,12,61.3,88,12,58.65,85,47,68.7,87,23,70.95,84,12,60.4,83,31,68.9,86,30,64.6,93,47,72.6,93,34,71.2,92,33,73.55,88,48,67.55,92,35,68.15,94,45,75.25,95,53,73.15,85,11,57.85,92,20,66.45,93,12,57.4,90,11,60.95,86,10,61.4,80,10,54.75,85,16,59.9,88,13,57.2,79,6,20.4,86,6,21.65,80,5,20.75,81,5,20.55,87,5,20.85


,TeamNum,Country,home showings,away showings,n showings,%shown,Rating_max,Rating_min,Rating_avg,Age_max,Age_min,Age_avg,Skill_Moves_max,Skill_Moves_min,Skill_Moves_avg,Ball_Control_max,Ball_Control_min,Ball_Control_avg,Dribbling_max,Dribbling_min,Dribbling_avg,Marking_max,Marking_min,Marking_avg,Sliding_Tackle_max,Sliding_Tackle_min,Sliding_Tackle_avg,Standing_Tackle_max,Standing_Tackle_min,Standing_Tackle_avg,Aggression_max,Aggression_min,Aggression_avg,Reactions_max,Reactions_min,Reactions_avg,Attacking_Position_max,Attacking_Position_min,Attacking_Position_avg,Interceptions_max,Interceptions_min,Interceptions_avg,Vision_max,Vision_min,Vision_avg,Composure_max,Composure_min,Composure_avg,Crossing_max,Crossing_min,Crossing_avg,Short_Pass_max,Short_Pass_min,Short_Pass_avg,Long_Pass_max,Long_Pass_min,Long_Pass_avg,Acceleration_max,Acceleration_min,Acceleration_avg,Speed_max,Speed_min,Speed_avg,Stamina_max,Stamina_min,Stamina_avg,Strength_max,Strength_min,Strength_avg,Balance_max,Balance_min,Balance_avg,Agility_max,Agility_min,Agility_avg,Jumping_max,Jumping_min,Jumping_avg,Heading_max,Heading_min,Heading_avg,Shot_Power_max,Shot_Power_min,Shot_Power_avg,Finishing_max,Finishing_min,Finishing_avg,Long_Shots_max,Long_Shots_min,Long_Shots_avg,Curve_max,Curve_min,Curve_avg,Freekick_Accuracy_max,Freekick_Accuracy_min,Freekick_Accuracy_avg,Penalties_max,Penalties_min,Penalties_avg,Volleys_max,Volleys_min,Volleys_avg,GK_Positioning_max,GK_Positioning_min,GK_Positioning_avg,GK_Diving_max,GK_Diving_min,GK_Diving_avg,GK_Kicking_max,GK_Kicking_min,GK_Kicking_avg,GK_Handling_max,GK_Handling_min,GK_Handling_avg,GK_Reflexes_max,GK_Reflexes_min,GK_Reflexes_avg
19,20,Brazil,30,34,64,5.203252,92,70,80.37037,36,19,27.185185,5,1,2.888889,95,21,71.333333,96,12,67.444444,90,9,56.481481,89,8,58.222222,91,12,61.592593,87,27,65.333333,88,67,78.925926,90,13,59.407407,91,11,60.888889,83,38,65.925926,84,31,70.259259,87,11,58.703704,83,15,69.259259,82,20,65.0,96,42,73.703704,91,35,74.333333,92,32,72.37037,88,49,70.851852,91,37,69.703704,96,30,72.333333,93,46,70.851852,84,12,59.333333,88,21,68.148148,89,11,53.666667,84,11,58.481481,81,12,57.555556,85,13,58.37037,81,17,55.962963,83,14,53.888889,78,3,17.222222,81,3,17.814815,82,3,17.259259,76,3,17.444444,87,3,17.888889


,TeamNum,Country,home showings,away showings,n showings,%shown,Rating_max,Rating_min,Rating_avg,Age_max,Age_min,Age_avg,Skill_Moves_max,Skill_Moves_min,Skill_Moves_avg,Ball_Control_max,Ball_Control_min,Ball_Control_avg,Dribbling_max,Dribbling_min,Dribbling_avg,Marking_max,Marking_min,Marking_avg,Sliding_Tackle_max,Sliding_Tackle_min,Sliding_Tackle_avg,Standing_Tackle_max,Standing_Tackle_min,Standing_Tackle_avg,Aggression_max,Aggression_min,Aggression_avg,Reactions_max,Reactions_min,Reactions_avg,Attacking_Position_max,Attacking_Position_min,Attacking_Position_avg,Interceptions_max,Interceptions_min,Interceptions_avg,Vision_max,Vision_min,Vision_avg,Composure_max,Composure_min,Composure_avg,Crossing_max,Crossing_min,Crossing_avg,Short_Pass_max,Short_Pass_min,Short_Pass_avg,Long_Pass_max,Long_Pass_min,Long_Pass_avg,Acceleration_max,Acceleration_min,Acceleration_avg,Speed_max,Speed_min,Speed_avg,Stamina_max,Stamina_min,Stamina_avg,Strength_max,Strength_min,Strength_avg,Balance_max,Balance_min,Balance_avg,Agility_max,Agility_min,Agility_avg,Jumping_max,Jumping_min,Jumping_avg,Heading_max,Heading_min,Heading_avg,Shot_Power_max,Shot_Power_min,Shot_Power_avg,Finishing_max,Finishing_min,Finishing_avg,Long_Shots_max,Long_Shots_min,Long_Shots_avg,Curve_max,Curve_min,Curve_avg,Freekick_Accuracy_max,Freekick_Accuracy_min,Freekick_Accuracy_avg,Penalties_max,Penalties_min,Penalties_avg,Volleys_max,Volleys_min,Volleys_avg,GK_Positioning_max,GK_Positioning_min,GK_Positioning_avg,GK_Diving_max,GK_Diving_min,GK_Diving_avg,GK_Kicking_max,GK_Kicking_min,GK_Kicking_avg,GK_Handling_max,GK_Handling_min,GK_Handling_avg,GK_Reflexes_max,GK_Reflexes_min,GK_Reflexes_avg
22,23,Germany,27,21,48,3.902439,92,77,83.26087,31,20,25.478261,4,1,2.652174,90,18,71.782609,87,10,67.521739,90,10,51.26087,91,10,53.782609,92,10,55.043478,89,29,62.130435,92,74,82.0,93,11,61.73913,89,20,59.478261,93,30,71.652174,91,48,75.391304,86,13,64.73913,90,25,74.347826,93,12,68.73913,92,38,69.173913,91,37,71.478261,90,35,70.217391,93,56,74.565217,83,33,62.521739,87,33,67.565217,82,34,67.782609,90,11,61.608696,87,22,69.695652,87,13,56.26087,88,10,61.478261,90,14,62.217391,87,11,55.086957,84,17,57.26087,88,11,58.565217,91,5,19.695652,89,6,20.521739,95,5,20.086957,90,6,20.0,89,5,20.695652


In [17]:
# Drop unnecessary features first
# tr.drop(columns=['tournament','city','country','neutral','date'],2)
tr = tr.reset_index()
print("Matches " + str(tr.shape[0]))
print("Features " + str(tr.shape[1]))
display(tr.head())

# For every game append the home_above_features vs away_above_features -> should give over 250 features to predict W T L 
add_home_away_maxminmean_vals_to_df(tr,extra_columns,True)
add_home_away_maxminmean_vals_to_df(tr,extra_columns,False)
print("Matches " + str(tr.shape[0]))
print("Features " + str(tr.shape[1]))
display(tr.head())

Matches 615
Features 13


,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_played,PT_DIFF,FTR
0,30613,2009-01-23,Costa Rica,Panama,3,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009,3,H
1,30629,2009-01-28,Mexico,Sweden,0,1,Friendly,Oakland,USA,True,2009,1,A
2,30638,2009-02-01,Costa Rica,Panama,0,0,UNCAF Cup,Tegucigalpa,Honduras,True,2009,0,T
3,30663,2009-02-11,France,Argentina,0,2,Friendly,Marseille,France,False,2009,2,A
4,30666,2009-02-11,Iran,Korea Republic,1,1,FIFA World Cup qualification,Teheran,Iran,False,2009,0,T


Matches 615
Features 235


,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_played,PT_DIFF,FTR,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
0,30613,

In [18]:
# Fill in with Data
rows = tr.shape[0]
print("rows: " + str(rows))
# display(tr.head(1))
counter = 0
# while counter < 1:
while counter < rows:

    #for each row check home_team
    ht = tr.loc[counter,'home_team'] # home team string (Costa Rica...)
    at = tr.loc[counter,'away_team']
    
    #get ht stats & at stats & both their indicies so we know what to pass on later
    ht_stats = explorer.loc[explorer['Country'] == ht]
    hti = int(ht_stats.TeamNum) - 1
    at_stats = explorer.loc[explorer['Country'] == at]
    ati = int(at_stats.TeamNum) - 1
    
    # append ht stats
    for stat in extra_columns:
        tr.loc[counter, 'ho_'+stat+'_max'] = ht_stats.loc[hti, stat+'_max']
        tr.loc[counter, 'ho_'+stat+'_min'] = ht_stats.loc[hti, stat+'_min']
        tr.loc[counter, 'ho_'+stat+'_avg'] = ht_stats.loc[hti, stat+'_avg'] 
        tr.loc[counter, 'aw_'+stat+'_max'] = at_stats.loc[ati, stat+'_max'] 
        tr.loc[counter, 'aw_'+stat+'_min'] = at_stats.loc[ati, stat+'_min'] 
        tr.loc[counter, 'aw_'+stat+'_avg'] = at_stats.loc[ati, stat+'_avg'] 
    
    
    
    counter += 1

display(tr.head())


rows: 615


,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year_played,PT_DIFF,FTR,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
0,30613,

In [19]:
# Save our CSV... it takes a while to build
tr.to_csv('true_result.csv', encoding='utf-8')

In [20]:
tr = pd.read_csv('true_result.csv')
# Visualising distribution of data
# from pandas.tools.ploting import scatter_matrix

# scatter_matrix(tr[["ho_Rating_max","ho_Rating_min","ho_Rating_avg","ho_Age_max","ho_Age_min","ho_Age_avg","ho_Skill_Moves_max","ho_Skill_Moves_min","ho_Skill_Moves_avg","ho_Ball_Control_max","ho_Ball_Control_min","ho_Ball_Control_avg","ho_Dribbling_max","ho_Dribbling_min","ho_Dribbling_avg","ho_Marking_max","ho_Marking_min","ho_Marking_avg","ho_Sliding_Tackle_max","ho_Sliding_Tackle_min","ho_Sliding_Tackle_avg","ho_Standing_Tackle_max","ho_Standing_Tackle_min","ho_Standing_Tackle_avg","ho_Aggression_max","ho_Aggression_min","ho_Aggression_avg","ho_Reactions_max","ho_Reactions_min","ho_Reactions_avg","ho_Attacking_Position_max","ho_Attacking_Position_min","ho_Attacking_Position_avg","ho_Interceptions_max","ho_Interceptions_min","ho_Interceptions_avg","ho_Vision_max","ho_Vision_min","ho_Vision_avg","ho_Composure_max","ho_Composure_min","ho_Composure_avg","ho_Crossing_max","ho_Crossing_min","ho_Crossing_avg","ho_Short_Pass_max","ho_Short_Pass_min","ho_Short_Pass_avg","ho_Long_Pass_max","ho_Long_Pass_min","ho_Long_Pass_avg","ho_Acceleration_max","ho_Acceleration_min","ho_Acceleration_avg","ho_Speed_max","ho_Speed_min","ho_Speed_avg","ho_Stamina_max","ho_Stamina_min","ho_Stamina_avg","ho_Strength_max","ho_Strength_min","ho_Strength_avg","ho_Balance_max","ho_Balance_min","ho_Balance_avg","ho_Agility_max","ho_Agility_min","ho_Agility_avg","ho_Jumping_max","ho_Jumping_min","ho_Jumping_avg","ho_Heading_max","ho_Heading_min","ho_Heading_avg","ho_Shot_Power_max","ho_Shot_Power_min","ho_Shot_Power_avg","ho_Finishing_max","ho_Finishing_min","ho_Finishing_avg","ho_Long_Shots_max","ho_Long_Shots_min","ho_Long_Shots_avg","ho_Curve_max","ho_Curve_min","ho_Curve_avg","ho_Freekick_Accuracy_max","ho_Freekick_Accuracy_min","ho_Freekick_Accuracy_avg","ho_Penalties_max","ho_Penalties_min","ho_Penalties_avg","ho_Volleys_max","ho_Volleys_min","ho_Volleys_avg","ho_GK_Positioning_max","ho_GK_Positioning_min","ho_GK_Positioning_avg","ho_GK_Diving_max","ho_GK_Diving_min","ho_GK_Diving_avg","ho_GK_Kicking_max","ho_GK_Kicking_min","ho_GK_Kicking_avg","ho_GK_Handling_max","ho_GK_Handling_min","ho_GK_Handling_avg","ho_GK_Reflexes_max","ho_GK_Reflexes_min","ho_GK_Reflexes_avg","aw_Rating_max","aw_Rating_min","aw_Rating_avg","aw_Age_max","aw_Age_min","aw_Age_avg","aw_Skill_Moves_max","aw_Skill_Moves_min","aw_Skill_Moves_avg","aw_Ball_Control_max","aw_Ball_Control_min","aw_Ball_Control_avg","aw_Dribbling_max","aw_Dribbling_min","aw_Dribbling_avg","aw_Marking_max","aw_Marking_min","aw_Marking_avg","aw_Sliding_Tackle_max","aw_Sliding_Tackle_min","aw_Sliding_Tackle_avg","aw_Standing_Tackle_max","aw_Standing_Tackle_min","aw_Standing_Tackle_avg","aw_Aggression_max","aw_Aggression_min","aw_Aggression_avg","aw_Reactions_max","aw_Reactions_min","aw_Reactions_avg","aw_Attacking_Position_max","aw_Attacking_Position_min","aw_Attacking_Position_avg","aw_Interceptions_max","aw_Interceptions_min","aw_Interceptions_avg","aw_Vision_max","aw_Vision_min","aw_Vision_avg","aw_Composure_max","aw_Composure_min","aw_Composure_avg","aw_Crossing_max","aw_Crossing_min","aw_Crossing_avg","aw_Short_Pass_max","aw_Short_Pass_min","aw_Short_Pass_avg","aw_Long_Pass_max","aw_Long_Pass_min","aw_Long_Pass_avg","aw_Acceleration_max","aw_Acceleration_min","aw_Acceleration_avg","aw_Speed_max","aw_Speed_min","aw_Speed_avg","aw_Stamina_max","aw_Stamina_min","aw_Stamina_avg","aw_Strength_max","aw_Strength_min","aw_Strength_avg","aw_Balance_max","aw_Balance_min","aw_Balance_avg","aw_Agility_max","aw_Agility_min","aw_Agility_avg","aw_Jumping_max","aw_Jumping_min","aw_Jumping_avg","aw_Heading_max","aw_Heading_min","aw_Heading_avg","aw_Shot_Power_max","aw_Shot_Power_min","aw_Shot_Power_avg","aw_Finishing_max","aw_Finishing_min","aw_Finishing_avg","aw_Long_Shots_max","aw_Long_Shots_min","aw_Long_Shots_avg","aw_Curve_max","aw_Curve_min","aw_Curve_avg","aw_Freekick_Accuracy_max","aw_Freekick_Accuracy_min","aw_Freekick_Accuracy_avg","aw_Penalties_max","aw_Penalties_min","aw_Penalties_avg","aw_Volleys_max","aw_Volleys_min","aw_Volleys_avg","aw_GK_Positioning_max","aw_GK_Positioning_min","aw_GK_Positioning_avg","aw_GK_Diving_max","aw_GK_Diving_min","aw_GK_Diving_avg","aw_GK_Kicking_max","aw_GK_Kicking_min","aw_GK_Kicking_avg","aw_GK_Handling_max","aw_GK_Handling_min","aw_GK_Handling_avg","aw_GK_Reflexes_max","aw_GK_Reflexes_min","aw_GK_Reflexes_avg"]], figsize=(10,10))
# scatter_matrix(tr[["ho_Rating_avg","ho_Age_avg","ho_Skill_Moves_avg","ho_Ball_Control_avg","ho_Dribbling_avg","ho_Marking_avg","ho_Sliding_Tackle_avg","ho_Standing_Tackle_avg","ho_Aggression_avg","ho_Reactions_avg","ho_Attacking_Position_avg","ho_Interceptions_avg","ho_Vision_avg","ho_Composure_avg"]], figsize=(10,10))


In [23]:
# del cols[:]
# del string_cols[:]
string_cols = ["home_team","away_team","neutral"]
cols = ["home_score","away_score","year_played","PT_DIFF"]
for stat in extra_columns:
    cols.append('ho_'+stat+'_max')
    cols.append('ho_'+stat+'_min')
    cols.append('ho_'+stat+'_avg')
    cols.append('aw_'+stat+'_max')
    cols.append('aw_'+stat+'_min')
    cols.append('aw_'+stat+'_avg')
print(len(cols))

226


In [26]:
# Seperate into feature set and target variable

x_all = tr.drop(['home_team','away_team','FTR','date','index','tournament','city','country','Unnamed: 0', 'neutral'],1)
y_all = tr['FTR']

# Standardize large to small & everyhting = somewhere nice
from sklearn.preprocessing import scale

#center to the mean & component wise unit variance
for col in cols:
    x_all[col] = scale(x_all[col])
    
display(x_all.head())

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


,home_score,away_score,year_played,PT_DIFF,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
0,1.261038,-0.948538,-1.597388,1.527258,0.121918,0.276780,-0.234223,-0.3341

In [27]:
def pre_process_features(X):
    '''Preprocess and turn footbal data into continuous vars'''
    #initalize
    output = pd.DataFrame(index = X.index)
    
    #investigate
    for col, col_data in X.iteritems():
        #if categorical convert to dummy vars
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
        
        #collect revision
        output = output.join(col_data)
    
    return output

x_all = pre_process_features(x_all)
display(x_all.head())
print("Processed feature columns ({} total features):\n{}".format(len(x_all.columns), list(x_all.columns))) 

,home_score,away_score,year_played,PT_DIFF,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
0,1.261038,-0.948538,-1.597388,1.527258,0.121918,0.276780,-0.234223,-0.3341

Processed feature columns (226 total features):
['home_score', 'away_score', 'year_played', 'PT_DIFF', 'ho_Rating_max', 'ho_Rating_min', 'ho_Rating_avg', 'ho_Age_max', 'ho_Age_min', 'ho_Age_avg', 'ho_Skill_Moves_max', 'ho_Skill_Moves_min', 'ho_Skill_Moves_avg', 'ho_Ball_Control_max', 'ho_Ball_Control_min', 'ho_Ball_Control_avg', 'ho_Dribbling_max', 'ho_Dribbling_min', 'ho_Dribbling_avg', 'ho_Marking_max', 'ho_Marking_min', 'ho_Marking_avg', 'ho_Sliding_Tackle_max', 'ho_Sliding_Tackle_min', 'ho_Sliding_Tackle_avg', 'ho_Standing_Tackle_max', 'ho_Standing_Tackle_min', 'ho_Standing_Tackle_avg', 'ho_Aggression_max', 'ho_Aggression_min', 'ho_Aggression_avg', 'ho_Reactions_max', 'ho_Reactions_min', 'ho_Reactions_avg', 'ho_Attacking_Position_max', 'ho_Attacking_Position_min', 'ho_Attacking_Position_avg', 'ho_Interceptions_max', 'ho_Interceptions_min', 'ho_Interceptions_avg', 'ho_Vision_max', 'ho_Vision_min', 'ho_Vision_avg', 'ho_Composure_max', 'ho_Composure_min', 'ho_Composure_avg', 'ho_Cross

In [41]:
from sklearn.cross_validation import train_test_split

#Shuffle & split the dataset into training & testing
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, 
                                                    test_size = 50,
                                                    random_state = 2,
                                                    stratify = y_all)
print(x_train.shape[0])
display(x_train.head(10))

print(x_test.shape[0])
display(x_test.head(10))

print(y_train.shape[0])
display(y_train.head(10))

print(y_test.shape[0])
display(y_test.head(10))


565


,home_score,away_score,year_played,PT_DIFF,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
504,-0.338010,-0.948538,1.007444,-0.238275,-0.301695,-0.687091,0.207005,1.5

50


,home_score,away_score,year_played,PT_DIFF,ho_Rating_max,ho_Rating_min,ho_Rating_avg,ho_Age_max,ho_Age_min,ho_Age_avg,ho_Skill_Moves_max,ho_Skill_Moves_min,ho_Skill_Moves_avg,ho_Ball_Control_max,ho_Ball_Control_min,ho_Ball_Control_avg,ho_Dribbling_max,ho_Dribbling_min,ho_Dribbling_avg,ho_Marking_max,ho_Marking_min,ho_Marking_avg,ho_Sliding_Tackle_max,ho_Sliding_Tackle_min,ho_Sliding_Tackle_avg,ho_Standing_Tackle_max,ho_Standing_Tackle_min,ho_Standing_Tackle_avg,ho_Aggression_max,ho_Aggression_min,ho_Aggression_avg,ho_Reactions_max,ho_Reactions_min,ho_Reactions_avg,ho_Attacking_Position_max,ho_Attacking_Position_min,ho_Attacking_Position_avg,ho_Interceptions_max,ho_Interceptions_min,ho_Interceptions_avg,ho_Vision_max,ho_Vision_min,ho_Vision_avg,ho_Composure_max,ho_Composure_min,ho_Composure_avg,ho_Crossing_max,ho_Crossing_min,ho_Crossing_avg,ho_Short_Pass_max,ho_Short_Pass_min,ho_Short_Pass_avg,ho_Long_Pass_max,ho_Long_Pass_min,ho_Long_Pass_avg,ho_Acceleration_max,ho_Acceleration_min,ho_Acceleration_avg,ho_Speed_max,ho_Speed_min,ho_Speed_avg,ho_Stamina_max,ho_Stamina_min,ho_Stamina_avg,ho_Strength_max,ho_Strength_min,ho_Strength_avg,ho_Balance_max,ho_Balance_min,ho_Balance_avg,ho_Agility_max,ho_Agility_min,ho_Agility_avg,ho_Jumping_max,ho_Jumping_min,ho_Jumping_avg,ho_Heading_max,ho_Heading_min,ho_Heading_avg,ho_Shot_Power_max,ho_Shot_Power_min,ho_Shot_Power_avg,ho_Finishing_max,ho_Finishing_min,ho_Finishing_avg,ho_Long_Shots_max,ho_Long_Shots_min,ho_Long_Shots_avg,ho_Curve_max,ho_Curve_min,ho_Curve_avg,ho_Freekick_Accuracy_max,ho_Freekick_Accuracy_min,ho_Freekick_Accuracy_avg,ho_Penalties_max,ho_Penalties_min,ho_Penalties_avg,ho_Volleys_max,ho_Volleys_min,ho_Volleys_avg,ho_GK_Positioning_max,ho_GK_Positioning_min,ho_GK_Positioning_avg,ho_GK_Diving_max,ho_GK_Diving_min,ho_GK_Diving_avg,ho_GK_Kicking_max,ho_GK_Kicking_min,ho_GK_Kicking_avg,ho_GK_Handling_max,ho_GK_Handling_min,ho_GK_Handling_avg,ho_GK_Reflexes_max,ho_GK_Reflexes_min,ho_GK_Reflexes_avg,aw_Rating_max,aw_Rating_min,aw_Rating_avg,aw_Age_max,aw_Age_min,aw_Age_avg,aw_Skill_Moves_max,aw_Skill_Moves_min,aw_Skill_Moves_avg,aw_Ball_Control_max,aw_Ball_Control_min,aw_Ball_Control_avg,aw_Dribbling_max,aw_Dribbling_min,aw_Dribbling_avg,aw_Marking_max,aw_Marking_min,aw_Marking_avg,aw_Sliding_Tackle_max,aw_Sliding_Tackle_min,aw_Sliding_Tackle_avg,aw_Standing_Tackle_max,aw_Standing_Tackle_min,aw_Standing_Tackle_avg,aw_Aggression_max,aw_Aggression_min,aw_Aggression_avg,aw_Reactions_max,aw_Reactions_min,aw_Reactions_avg,aw_Attacking_Position_max,aw_Attacking_Position_min,aw_Attacking_Position_avg,aw_Interceptions_max,aw_Interceptions_min,aw_Interceptions_avg,aw_Vision_max,aw_Vision_min,aw_Vision_avg,aw_Composure_max,aw_Composure_min,aw_Composure_avg,aw_Crossing_max,aw_Crossing_min,aw_Crossing_avg,aw_Short_Pass_max,aw_Short_Pass_min,aw_Short_Pass_avg,aw_Long_Pass_max,aw_Long_Pass_min,aw_Long_Pass_avg,aw_Acceleration_max,aw_Acceleration_min,aw_Acceleration_avg,aw_Speed_max,aw_Speed_min,aw_Speed_avg,aw_Stamina_max,aw_Stamina_min,aw_Stamina_avg,aw_Strength_max,aw_Strength_min,aw_Strength_avg,aw_Balance_max,aw_Balance_min,aw_Balance_avg,aw_Agility_max,aw_Agility_min,aw_Agility_avg,aw_Jumping_max,aw_Jumping_min,aw_Jumping_avg,aw_Heading_max,aw_Heading_min,aw_Heading_avg,aw_Shot_Power_max,aw_Shot_Power_min,aw_Shot_Power_avg,aw_Finishing_max,aw_Finishing_min,aw_Finishing_avg,aw_Long_Shots_max,aw_Long_Shots_min,aw_Long_Shots_avg,aw_Curve_max,aw_Curve_min,aw_Curve_avg,aw_Freekick_Accuracy_max,aw_Freekick_Accuracy_min,aw_Freekick_Accuracy_avg,aw_Penalties_max,aw_Penalties_min,aw_Penalties_avg,aw_Volleys_max,aw_Volleys_min,aw_Volleys_avg,aw_GK_Positioning_max,aw_GK_Positioning_min,aw_GK_Positioning_avg,aw_GK_Diving_max,aw_GK_Diving_min,aw_GK_Diving_avg,aw_GK_Kicking_max,aw_GK_Kicking_min,aw_GK_Kicking_avg,aw_GK_Handling_max,aw_GK_Handling_min,aw_GK_Handling_avg,aw_GK_Reflexes_max,aw_GK_Reflexes_min,aw_GK_Reflexes_avg
569,-1.137535,-0.075258,1.379563,-0.238275,0.827940,0.276780,0.144746,-0.33

565


504    H
24     T
352    A
550    T
351    H
536    A
161    T
229    H
1      A
21     H
Name: FTR, dtype: object

50


569    A
568    H
294    H
350    H
296    T
328    H
271    H
47     H
13     H
127    H
Name: FTR, dtype: object

# Trainig & Evaluating Models

In [42]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, targets):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    display(targets.head(5))
    display(y_pred.head(5))
    f1 = f1_score(targets, y_pred, labels='H', average='micro')
    print("And now f1 below.")
    print(f1)
    print(type(f1))
    print("")
    return(f1, sum(targets == y_pred) / float(len(y_pred)))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

# LOGISTIC REGRESSION

Used to predict probability of occurance of event.

In [43]:
# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 11, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, x_train, y_train, x_test, y_test)
print ('')
train_predict(clf_B, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_C, x_train, y_train, x_test, y_test)
print('')

Training a LogisticRegression using a training set size of 565. . .
Trained model in 0.1379 seconds
Made predictions in 0.0009 seconds.


504    H
24     T
352    A
550    T
351    H
Name: FTR, dtype: object

AttributeError: 'numpy.ndarray' object has no attribute 'head'